In [1]:
pacman::p_load(
    Seurat, SeuratDisk, tidyverse
)

## Requirement
Cell annotation file: Indicates cell type information which placed in `meta/annotations/{sample_name}.tsv`

In [24]:
list.files(
    "../../meta/annotations/",
    pattern = ".tsv"
) %>%
as.list() %>%
lapply(
    function(fpath) {
        fname <- str_split(fpath, "\\.", simplify = TRUE)[[1]]
        seurat <-
            LoadH5Seurat(
                "../../resources/",
                fname,
                ".h5seurat"
            ) %>%
            SingleCellPipeline::assign_celltype(
                annotation = read_tsv(
                    paste0(
                        "../../meta/annotations/",
                        fname,
                        ".tsv"
                    )
                ),
                cell.meta = read_tsv(
                    "../../meta/cell_metadata.tsv",
                    show_col_types = FALSE
                ),
                sample.meta = read_tsv(
                    "../../meta/sample_metadata.tsv",
                    show_col_types = FALSE
                )
            )
        seurat@meta.data %>%
            tibble::rownames_to_column("cell") %>%
            select(
                cell, celltype, color.celltype, group, marker, color.sample
            ) %>%
            mutate(
                sample = fname
            ) %>%
            return()
    }
) %>%
bind_rows() %>%
write_tsv(
    "../../resources/cellinfo.ci"
)